In [1]:
import numpy as np
from copulas.univariate import GaussianKDE 
from copulas.bivariate import Clayton
from scipy.stats import t
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from collections import OrderedDict
import pandas as pd
import random
from scipy.stats import genpareto
from scipy.stats import expon,poisson,pareto
import pathlib
from scipy import integrate 
from Utils_StochasticSimulation import *

C:\Users\nmadhar\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
sns.set_theme(style='white')
alphaVaR = 0.9975
alphaU = 0.85

---
*This nooteook gives an illustration of TRMs empirical estimation on a single sample generated with our joint simulation algorithms. We recommend the esitmation on multiple sample (running the joint simulate multiple times) for accurate TRMs estimates.*

---

# I. Weak Asymptotic Dependence 
## 1. Data preparation 

In [3]:
path = r'C:\Users\nmadhar\Desktop\GANEV\SimulationOfExtremeScenarios\Data\DataK3ExtremalCoefficient\Gumbel33'
name =  'Gumbel33_1'#'Gumbel60_2' utilisé pour la calibration du M 
chiCopula = 1/3

    
filename = name
Data = pd.read_excel(pathlib.PurePath(path, filename + '.xlsx')) 
nu1,nu2,nu3= 2,3,2.5
X = (t.ppf(Data.values[:,0],df=nu1)).reshape(-1,1)
Y = (t.ppf(Data.values[:,1],df=nu2)).reshape(-1,1)
Z = ((t.ppf(Data.values[:,2],df=nu3))).reshape(-1,1)
df = pd.DataFrame(np.concatenate([X,Y,Z],axis=1))
n,K = df.shape

VaRTheo = t.ppf(alphaVaR,df=nu1)
ESEmp = np.mean(X[X>VaRTheo]) 


X0 = (-np.log(1-Data.iloc[:,0].values)).reshape(-1,1)
Y0 = (-np.log(1-Data.iloc[:,1].values)).reshape(-1,1)
Z0 = (-np.log(1-Data.iloc[:,2].values)).reshape(-1,1)
dfStand = pd.DataFrame(np.concatenate([X0,Y0,Z0],axis=1))

uX0,uY0,uZ0 = np.quantile(X0,alphaU),np.quantile(Y0,alphaU),np.quantile(Z0,alphaU)

print(f'Gamma of X0 = %.2f'%(genpareto.fit(X0[X0>uX0]-uX0)[0]))
print(f'Gamma of Y0 = %.2f'%(genpareto.fit(Y0[Y0>uY0]-uY0)[0]))
print(f'Gamma of Z0 = %.2f'%(genpareto.fit(Z0[Z0>uZ0]-uZ0)[0]))



TX0 = (np.arange(n)[dfStand.iloc[:,0]>uX0]).reshape(-1,)
TY0 = (np.arange(n)[dfStand.iloc[:,1]>uY0]).reshape(-1,)
TZ0 = (np.arange(n)[dfStand.iloc[:,2]>uZ0]).reshape(-1,)

T0 = np.concatenate([TX0,TY0,TZ0])

T0 = np.array(list(OrderedDict.fromkeys(list(T0))),dtype=int)
ExcessStand = dfStand.iloc[T0,:]
ExcessStand.iloc[:,0] = ExcessStand.iloc[:,0] - uX0
ExcessStand.iloc[:,1] = ExcessStand.iloc[:,1] - uY0
ExcessStand.iloc[:,2] = ExcessStand.iloc[:,2] - uZ0

ExcessStand = ExcessStand.values

Gamma of X0 = 0.01
Gamma of Y0 = -0.08
Gamma of Z0 = -0.01


C:\Users\nmadhar\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


In [4]:
VaRTheoX, ESTheoX, DCTETheoX,MESTheoX = getRiskMeasuresStudentGumbelK3(alphaVaR,nuVect=[nu1,nu2,nu3],theta=np.log(2)/np.log(2-chiCopula))
VaRTheoY, ESTheoY, DCTETheoY,MESTheoY = getRiskMeasuresStudentGumbelK3(alphaVaR,nuVect=[nu2,nu1,nu3],theta=np.log(2)/np.log(2-chiCopula))
VaRTheoZ, ESTheoZ, DCTETheoZ,MESTheoZ = getRiskMeasuresStudentGumbelK3(alphaVaR,nuVect=[nu3,nu1,nu2],theta=np.log(2)/np.log(2-chiCopula))
ParamsOriginScale = [nu1,uX0,nu2,uY0,nu3,uZ0 ]
VaR = [VaRTheoX,VaRTheoY,VaRTheoZ]
VaREmpX, ESEmpX, DCTEEmpX,Nb_ExcDepX,MESEmpX,Nb_ExcMESX = getRiskMeasuresEmpiricalMES(df,0,alphaVaR,ExcdTheo=True,VaRRF= [VaRTheoX,VaRTheoY,VaRTheoZ])

In [5]:
TX = (np.arange(n)[df.iloc[:,0]>VaRTheoX]).reshape(-1,)
TY = (np.arange(n)[df.iloc[:,1]>VaRTheoY]).reshape(-1,)
TZ = (np.arange(n)[df.iloc[:,2]>VaRTheoZ]).reshape(-1,)

T = np.concatenate([TX,TY,TZ])

T = np.array(list(OrderedDict.fromkeys(list(T))),dtype=int)
Excess = df.iloc[T,:]

## 2. One-shoot estimation of TRMs

In [6]:
myseed = 1841818
ESExtX,ESSimuX,Nb_VaRX = StabilityNewSamples(ExcessStand,X,[nu1,uX0],VaRTheoX,M=10000,K=1,seed=myseed)
SummaryErrorEmpSimuExt(ESEmpX,ESSimuX,ESExtX,ESTheoX,(X>VaRTheoX).sum(),Nb_VaRX,'ES')

----------ES----------
Relative error on empirical estimation -0.18 
Empirical estimation 23.15, Theoretical value 28.25 
Nb. of Observations above VaR Level 6 on original sample

Relative error of empirical estimation on simulated sample 0.004 
Empirical estimation on simulated sample 28.37, Theoretical value 28.25 
Nb. of Observations above VaR Level 84 on simulated sample

Relative error of empirical estimation on extended sample -0.008 
Empirical estimation on extended sample 28.03, Theoretical value 28.25 


In [7]:
DCTE_SimuX,DCTE_ExtX,DCTE_SimuY,DCTE_ExtY,DCTE_SimuZ,DCTE_ExtZ,Nb_DCTE = StabilityonlyDCTE_NewSamples(ExcessStand,Excess,ParamsOriginScale,VaR,M=10000,K=1,seed=myseed)
SummaryErrorEmpSimuExt(DCTEEmpX,DCTE_SimuX,DCTE_ExtX,DCTETheoX,Nb_ExcDepX,Nb_DCTE,'DCTE')

----------DCTE----------
Relative error on empirical estimation 0.11 
Empirical estimation 48.82, Theoretical value 44.16 
Nb. of Observations above VaR Level 1 on original sample

Relative error of empirical estimation on simulated sample 0.022 
Empirical estimation on simulated sample 45.11, Theoretical value 44.16 
Nb. of Observations above VaR Level 29 on simulated sample

Relative error of empirical estimation on extended sample 0.024 
Empirical estimation on extended sample 45.24, Theoretical value 44.16 


In [8]:
MES_SimuX,MES_ExtX,MES_SimuY,MES_ExtY,MES_SimuZ,MES_ExtZ,Nb_MES = StabilityonlyMES_NewSamples(ExcessStand,Excess,ParamsOriginScale,VaR,M=10000,K=1,seed=myseed)
SummaryErrorEmpSimuExt(MESEmpX,MES_SimuX,MES_ExtX,MESTheoX,Nb_ExcDepX,Nb_MES,'MES')

----------MES----------
Relative error on empirical estimation 0.39 
Empirical estimation 48.82, Theoretical value 35.24 
Nb. of Observations above VaR Level 1 on original sample

Relative error of empirical estimation on simulated sample 0.047 
Empirical estimation on simulated sample 36.90, Theoretical value 35.24 
Nb. of Observations above VaR Level 38 on simulated sample

Relative error of empirical estimation on extended sample 0.056 
Empirical estimation on extended sample 37.21, Theoretical value 35.24 


# II. Strong Asymptotic Dependence 

## 1. Data Preparation

In [9]:
path = r'C:\Users\nmadhar\Desktop\GANEV\SimulationOfExtremeScenarios\Data\DataK3ExtremalCoefficient\Gumbel90'
name =  'Gumbel90_1'#'Gumbel60_2' utilisé pour la calibration du M 
chiCopula = 0.9

    
filename = name
Data = pd.read_excel(pathlib.PurePath(path, filename + '.xlsx')) 
nu1,nu2,nu3= 2,3,2.5
X = (t.ppf(Data.values[:,0],df=nu1)).reshape(-1,1)
Y = (t.ppf(Data.values[:,1],df=nu2)).reshape(-1,1)
Z = ((t.ppf(Data.values[:,2],df=nu3))).reshape(-1,1)
df = pd.DataFrame(np.concatenate([X,Y,Z],axis=1))
n,K = df.shape

VaRTheo = t.ppf(alphaVaR,df=nu1)
ESEmp = np.mean(X[X>VaRTheo]) 


X0 = (-np.log(1-Data.iloc[:,0].values)).reshape(-1,1)
Y0 = (-np.log(1-Data.iloc[:,1].values)).reshape(-1,1)
Z0 = (-np.log(1-Data.iloc[:,2].values)).reshape(-1,1)
dfStand = pd.DataFrame(np.concatenate([X0,Y0,Z0],axis=1))

uX0,uY0,uZ0 = np.quantile(X0,alphaU),np.quantile(Y0,alphaU),np.quantile(Z0,alphaU)

print(f'Gamma of X0 = %.2f'%(genpareto.fit(X0[X0>uX0]-uX0)[0]))
print(f'Gamma of Y0 = %.2f'%(genpareto.fit(Y0[Y0>uY0]-uY0)[0]))
print(f'Gamma of Z0 = %.2f'%(genpareto.fit(Z0[Z0>uZ0]-uZ0)[0]))



TX0 = (np.arange(n)[dfStand.iloc[:,0]>uX0]).reshape(-1,)
TY0 = (np.arange(n)[dfStand.iloc[:,1]>uY0]).reshape(-1,)
TZ0 = (np.arange(n)[dfStand.iloc[:,2]>uZ0]).reshape(-1,)

T0 = np.concatenate([TX0,TY0,TZ0])

T0 = np.array(list(OrderedDict.fromkeys(list(T0))),dtype=int)
ExcessStand = dfStand.iloc[T0,:]
ExcessStand.iloc[:,0] = ExcessStand.iloc[:,0] - uX0
ExcessStand.iloc[:,1] = ExcessStand.iloc[:,1] - uY0
ExcessStand.iloc[:,2] = ExcessStand.iloc[:,2] - uZ0

ExcessStand = ExcessStand.values

Gamma of X0 = -0.01
Gamma of Y0 = -0.05
Gamma of Z0 = -0.03


In [10]:
VaRTheoX, ESTheoX, DCTETheoX,MESTheoX = getRiskMeasuresStudentGumbelK3(alphaVaR,nuVect=[nu1,nu2,nu3],theta=np.log(2)/np.log(2-chiCopula))
VaRTheoY, ESTheoY, DCTETheoY,MESTheoY = getRiskMeasuresStudentGumbelK3(alphaVaR,nuVect=[nu2,nu1,nu3],theta=np.log(2)/np.log(2-chiCopula))
VaRTheoZ, ESTheoZ, DCTETheoZ,MESTheoZ = getRiskMeasuresStudentGumbelK3(alphaVaR,nuVect=[nu3,nu1,nu2],theta=np.log(2)/np.log(2-chiCopula))
ParamsOriginScale = [nu1,uX0,nu2,uY0,nu3,uZ0 ]
VaR = [VaRTheoX,VaRTheoY,VaRTheoZ]
VaREmpX, ESEmpX, DCTEEmpX,Nb_ExcDepX,MESEmpX,Nb_ExcMESX = getRiskMeasuresEmpiricalMES(df,0,alphaVaR,ExcdTheo=True,VaRRF= [VaRTheoX,VaRTheoY,VaRTheoZ])

In [11]:
TX = (np.arange(n)[df.iloc[:,0]>VaRTheoX]).reshape(-1,)
TY = (np.arange(n)[df.iloc[:,1]>VaRTheoY]).reshape(-1,)
TZ = (np.arange(n)[df.iloc[:,2]>VaRTheoZ]).reshape(-1,)

T = np.concatenate([TX,TY,TZ])

T = np.array(list(OrderedDict.fromkeys(list(T))),dtype=int)
Excess = df.iloc[T,:]

In [12]:
TX = (np.arange(n)[df.iloc[:,0]>VaRTheoX]).reshape(-1,)
TY = (np.arange(n)[df.iloc[:,1]>VaRTheoY]).reshape(-1,)
TZ = (np.arange(n)[df.iloc[:,2]>VaRTheoZ]).reshape(-1,)

T = np.concatenate([TX,TY,TZ])

T = np.array(list(OrderedDict.fromkeys(list(T))),dtype=int)
Excess = df.iloc[T,:]

## 2. One-shoot estimation of TRMs

In [17]:
myseed = 1841812
ESExtX,ESSimuX,Nb_VaRX = StabilityNewSamples(ExcessStand,X,[nu1,uX0],VaRTheoX,M=10000,K=1,seed=myseed)
SummaryErrorEmpSimuExt(ESEmpX,ESSimuX,ESExtX,ESTheoX,(X>VaRTheoX).sum(),Nb_VaRX,'ES')

----------ES----------
Relative error on empirical estimation 0.34 
Empirical estimation 37.90, Theoretical value 28.25 
Nb. of Observations above VaR Level 2 on original sample

Relative error of empirical estimation on simulated sample -0.072 
Empirical estimation on simulated sample 26.22, Theoretical value 28.25 
Nb. of Observations above VaR Level 132 on simulated sample

Relative error of empirical estimation on extended sample -0.066 
Empirical estimation on extended sample 26.39, Theoretical value 28.25 


In [18]:
DCTE_SimuX,DCTE_ExtX,DCTE_SimuY,DCTE_ExtY,DCTE_SimuZ,DCTE_ExtZ,Nb_DCTE = StabilityonlyDCTE_NewSamples(ExcessStand,Excess,ParamsOriginScale,VaR,M=10000,K=1,seed=myseed)
SummaryErrorEmpSimuExt(DCTEEmpX,DCTE_SimuX,DCTE_ExtX,DCTETheoX,Nb_ExcDepX,Nb_DCTE,'DCTE')

----------DCTE----------
Relative error on empirical estimation 0.25 
Empirical estimation 37.90, Theoretical value 30.29 
Nb. of Observations above VaR Level 2 on original sample

Relative error of empirical estimation on simulated sample -0.091 
Empirical estimation on simulated sample 27.55, Theoretical value 30.29 
Nb. of Observations above VaR Level 118 on simulated sample

Relative error of empirical estimation on extended sample -0.085 
Empirical estimation on extended sample 27.72, Theoretical value 30.29 


In [19]:
MES_SimuX,MES_ExtX,MES_SimuY,MES_ExtY,MES_SimuZ,MES_ExtZ,Nb_MES = StabilityonlyMES_NewSamples(ExcessStand,Excess,ParamsOriginScale,VaR,M=10000,K=1,seed=myseed)
SummaryErrorEmpSimuExt(MESEmpX,MES_SimuX,MES_ExtX,MESTheoX,Nb_ExcDepX,Nb_MES,'MES')

----------MES----------
Relative error on empirical estimation 0.28 
Empirical estimation 37.90, Theoretical value 29.60 
Nb. of Observations above VaR Level 2 on original sample

Relative error of empirical estimation on simulated sample -0.085 
Empirical estimation on simulated sample 27.09, Theoretical value 29.60 
Nb. of Observations above VaR Level 122 on simulated sample

Relative error of empirical estimation on extended sample -0.079 
Empirical estimation on extended sample 27.26, Theoretical value 29.60 
